# Code for Pulling Lyrics from Genius

In [11]:
from lyricsgenius import Genius
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import sqlite3
conn = sqlite3.connect('../datasets/lyrics.db')
from requests.exceptions import Timeout

In [2]:
#Lyrcis
genius = Genius(os.getenv('GENIUS_KEY'))
genius.timeout = 15
genius.remove_section_headers = True

#need to write function to generate for each and store in a datastructure to not duplicate searches.

In [3]:
#final_df = pd.read_csv("../datasets/Final database.csv")
full_df = pd.read_csv("../datasets/Database to calculate popularity.csv")


In [4]:
holder_df = full_df[full_df['country']=='USA']
holder_df.head()

,Unnamed: 0,country,date,position,uri,track,title,artist
280200,280200,USA,05/11/2020,1.0,https://open.spotify.com/track/35mvY5S1H3J2QZy...,"positions , Ariana Grande",positions,Ariana Grande
280201,280201,USA,05/11/2020,2.0,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,"Lemonade , Internet Money",Lemonade,Internet Money
280202,280202,USA,05/11/2020,3.0,https://open.spotify.com/track/6Im9k8u9iIzKMrm...,"34+35 , Ariana Grande",34+35,Ariana Grande
280203,280203,USA,05/11/2020,4.0,https://open.spotify.com/track/47EiUVwUp4C9fGc...,"Dakiti , Bad Bunny, Jhay Cortez",Dakiti,"Bad Bunny, Jhay Cortez"
280204,280204,USA,05/11/2020,5.0,https://open.spotify.com/track/3tjFYV6RSFtuktY...,"Mood , 24kGoldn",Mood,24kGoldn


In [7]:
def storeLyrics(songName,artistName,uri,cursor): #sep and local
    artist = genius.search_artist(artistName, max_songs=0)
    if artist==None:
        return None
    song = genius.search_song(songName, artist.name)
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','')
    query = 'INSERT INTO lyrics(song,artist,lyric,uri) VALUES (\"'+songName+'\",\"'+str(artistName.split(','))+'\",\"'+lyrics.replace('\"','\'')+'\",\''+str(uri)+'\')'
    cursor.execute(query)
    conn.commit()
def test_store(songName,artistName):
    artist = genius.search_artist(artistName, max_songs=0)
    if artist==None:
        return None
    song = genius.search_song(songName, artist.name)
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','')

In [12]:
cur = conn.cursor()
cur.execute("CREATE TABLE lyrics (song text NOT NULL, artist text NOT NULL, lyric text NOT NULL, uri text NOT NULL);")
taken = []
count = 0
for index,row in holder_df.iterrows():
    retries = 0
    if (row['uri'] in taken):
        pass
    else:
        while retries<3:
            try:
                storeLyrics(row['title'],row['artist'],row['uri'],cur)
                taken.append(row['uri'])
                retries = 3
            except Timeout as e:
                retries += 1
                continue
    count = index

#conn.close()

Searching for songs by  Ariana Grande ...

Changing artist name to 'Ariana Grande'
Done. Found 0 songs.
Searching for " positions " by Ariana Grande...
Done.
Searching for songs by  Internet Money ...

Changing artist name to 'Internet Money'
Done. Found 0 songs.
Searching for " Lemonade  " by Internet Money...
Done.
Searching for songs by  Ariana Grande ...

Changing artist name to 'Ariana Grande'
Done. Found 0 songs.
Searching for " 34+35 " by Ariana Grande...
Done.
Searching for songs by  Bad Bunny, Jhay Cortez ...

Changing artist name to 'Bad Bunny & Jhay Cortez'
Done. Found 0 songs.
Searching for " Dakiti " by Bad Bunny & Jhay Cortez...
Done.
Searching for songs by  24kGoldn ...

Changing artist name to '24kGoldn'
Done. Found 0 songs.
Searching for " Mood  " by 24kGoldn...
Done.
Searching for songs by  Pop Smoke ...

Changing artist name to 'Pop Smoke'
Done. Found 0 songs.
Searching for " For The Night  " by Pop Smoke...
Done.
Searching for songs by  Cardi B ...

Changing artist 

OperationalError: near "Boris": syntax error

In [13]:
#Sing testing
#test_store('Diamonds','Sam Smith')
count


281014

In [ ]:
conn.commit()
conn.close()